In [1]:
import pandas as pd
import numpy as numpy
import json 
import os 
import pylab as pl
import urllib
import requests
import geopandas as gp
import fiona
import shapely
from fiona.crs import from_epsg
import os
import pandas as pd
import urllib
import requests
import json
import sys
import io
import shapely
import base64
import urllib.request
import functools
import geopandas as gp
import fiona
import numpy as np
import matplotlib.pylab as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [ ]:
url = 'https://data.cityofnewyork.us/api/views/8h9b-rp9u/rows.csv?accessType=DOWNLOAD'
arrests = pd.read_csv(url)
arrests.head()

In [ ]:
violent = [139,140,153,155,157,164,166,168,175,176
          ,177,178,179,180,183,185,187,261,263,379
           ,380,397,586,639,640,665,104,105,106,
          107,109,112,122,123,124,125,129]
qol = [789,566,258,610, 612, 614, 616, 622, 803, 862]

violations = 789,566,258,604,617,600,602,610, 612, 614, 616, 622, 803, 862,625
firework possession, marijuana possession,possession of graffiti instruments,loitering -use control substance,
loitering-prostitution, loitering-sexual deviation, loitering-gambling in a public place, 
, in public under influence of narcotics, alcohol under 21, public consumption of alc, unreasonable noise

In [ ]:
arrests['year'] = arrests['ARREST_DATE'].str[-2:]
arrests.head()

In [ ]:
arrests = arrests[(arrests['PD_CD'].isin(violent))|(arrests['PD_CD'].isin(qol))]

In [ ]:
arrests.shape

In [ ]:
arrests.loc[arrests['PD_CD'].isin(violent), 'category'] = 'vio'
arrests.loc[arrests['PD_CD'].isin(qol), 'category'] = 'qol'

In [ ]:
arrests.head()

In [ ]:
grouped = arrests.groupby(['ARREST_PRECINCT','category']).count().reset_index()
grouped.head()

In [ ]:
grouped = grouped[['ARREST_PRECINCT','category','ARREST_KEY']]

In [ ]:
grouped = grouped.pivot(index='ARREST_PRECINCT', columns='category', values='ARREST_KEY')
grouped.head()

In [ ]:
grouped['qol_vio_ratio'] = grouped['qol']/grouped['vio']

grouped.describe()

In [ ]:
#import precinct shapefile

url = 'https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=Shapefile'
precinct = gp.GeoDataFrame.from_file(url)
precinct.head(2)

In [ ]:
#join to model data

precinct = precinct.merge(grouped,left_on='precinct', right_on='ARREST_PRECINCT')

precinct.shape

In [ ]:
path = '/Users/katharinevoorhees/patrol_allocation/precinct_population_dayadjust.csv'

pop = pd.read_csv(path)

pop.head()

In [ ]:
precinct = precinct.merge(pop, on='precinct')

In [ ]:
precinct.head()

In [ ]:
precinct['percapita_day'] = (precinct['qol']/\
        precinct['adjusted_day_precinct_x'])*10000

In [ ]:
precinct = precinct[precinct.precinct != 22]

In [ ]:
precinct.to_crs(epsg=2263, inplace=True)

In [ ]:
 #make cloropleth of precincts shaded by allocation in this model 

f,ax = plt.subplots(figsize=(10,10))
precinct.plot(column='percapita_day', cmap="Blues", edgecolor="grey",ax=ax,
              scheme='quantiles',k=10, lw=.5, legend=True)
ax.axis('off')
plt.title('Ratio' ,fontsize=(20))

leg = ax.get_legend()
leg.set_bbox_to_anchor((0,-.1,.25,1));

In [ ]:
precinct.sort_values('qol_vio_ratio', ascending=False)

In [ ]:
path = '/Users/katharinevoorhees/Downloads/stopfrisk_noarrest_rate.csv'
stopfrisk = pd.read_csv(path)

In [ ]:
stopfrisk.head()

In [ ]:
stopfrisk['2018_no_rate'].hist(bins=30)

In [ ]:
precinct = precinct.merge(stopfrisk) 
precinct.head()

In [ ]:
precinct['switch'] = 1 - precinct['2018_no_rate']

In [ ]:
#make cloropleth of precincts shaded by allocation in this model 

f,ax = plt.subplots(figsize=(10,10))
precinct.plot(column='2018_no_rate', cmap="Blues", edgecolor="grey",ax=ax,
              scheme='quantiles',k=10, lw=.5, legend=True)
ax.axis('off')
plt.title('Ratio' ,fontsize=(20))

leg = ax.get_legend()
leg.set_bbox_to_anchor((0,-.1,.25,1));

In [ ]:
#look specifically at stop and frisks for suspicion of un-super-scary crimes